In [2]:
%reload_ext autoreload
%autoreload 3

import mne
import numpy as np
from src.EEGNet.preprocessing import utils
from pathlib import Path
import re
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, robust_scale

# data directories
data_path = f'/Volumes/Extreme SSD/PhD/MPI-LEMON/EEG_Raw_BIDS_ID/'
mpi_path = f'/Volumes/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/'

ModuleNotFoundError: No module named 'simsimd'

### Which data are excluded from MPI dataset for processing?

In [2]:
# get all subject ids in preprocessed folder
sub_ids_pro = []
for subject_path in sorted(Path(mpi_path).glob('sub-*')):
  sub_ids_pro.append(re.search('sub-(.*)_(.*)', subject_path.stem).group(1))
sub_ids_pro = set(sub_ids_pro)  # drop duplicates

# get all subject ids in raw folder
sub_ids_raw = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_ids_raw.append(re.search('sub-(.*)', subject_path.stem).group(1))
sub_ids_raw = set(sub_ids_raw)  # drop any possible duplicates

# find any ids in sub_ids_raw that are not in sub_ids_pro
sub_ids_excluded = sorted(list(sub_ids_pro - sub_ids_raw))

## TODO: delete the entire cell: Annotations & Data Segmentation
### Data with different annotation patterns

In [124]:
fnames_annot_misbehave = {}
sub_ids_raw = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_id = subject_path.stem
  sub_ids_raw.append(subject_path.stem)
  raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)
  
  # check annot names
  points = utils.annotations_checker(raw.annotations.description.copy())

  # save those with deflection
  if sum(points.values()) != 2:
     fnames_annot_misbehave[sub_id] = points

### 1. segment and preprocess data with common annotation patterns

In [136]:
sub_ids_normal = sorted(list(set(sub_ids_raw) - set(fnames_annot_misbehave.keys())))
output_path = '/Volumes/Extreme SSD/PhD/MPI-LEMON/EEG_Raw_segmented/'
rsfreq = 250

In [138]:
for sub_id in sub_ids_normal[:5]:
    raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)
    onsets, pattern = utils.find_pattern(raw)
    raws = utils.segment_raw(raw, onsets, pattern, duration=60)

    # save EC/EO segments in brainvision format
    ## create subject folder
    subject_path = output_path + sub_id
    Path(subject_path).mkdir(parents=True, exist_ok=True)
    ## downsample and save
    for k in raws.keys():
        raws[k].resample(rsfreq)
        raws[k].export(subject_path + f'/{sub_id}_{k}.vhdr')

### 2. segment raw data with not common annotation patterns

In [ ]:
typ1 = ['sub-010026']
# an extera ''New Segment/', 'Comment/no USB Connection to actiCAP' markers in the beginning.

typ2 = ['sub-010030']
#an extera ''New Segment/', 'Comment/no USB Connection to actiCAP' markers in the beginning without extra switch markers.

typ3 = ['sub-010062', 'sub-010064', 'sub-010134'] # only one switch markers in the beginning 

typ4 = ['sub-010191'] # an extra 'New Segment/' 'Comment/no USB Connection to actiCAP' 'Stimulus/S  1' markers in the beginning.

typ5 = ['sub-010264'] # no switch markers in the beginning

# ----------------------------------------------
typ6 = ['sub-010126']
# this subject there is no switch markers, and the annotation for eye closed is 'Stimulus/S208'

# no switch markers in the annotations, and there is only two useless markers in the beginning
typ7 = ['sub-010138', 'sub-010155', 'sub-010157',
        'sub-010162', 'sub-010163', 'sub-010164', 'sub-010165',
        'sub-010166', 'sub-010168', 'sub-010228', 'sub-010233',
        'sub-010239', 'sub-010255', 'sub-010257', 'sub-010258',
        'sub-010260', 'sub-010261', 'sub-010262', 'sub-010263',
        'sub-010267', 'sub-010268', 'sub-010269', 'sub-010270',
        'sub-010271', 'sub-010272', 'sub-010273', 'sub-010274',
        'sub-010275', 'sub-010284', 'sub-010311', 'sub-010315',
        'sub-010318']

## Divide data to eye closed and eye open segments

In [ ]:
# Check if all the switch markers's onsets are the same as the beginning of the EC/EO segments
sanity_check = {}
all_eo = []
all_ec = []
subject_ids = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
    sub_id = subject_path.stem
    subject_ids.append(sub_id)
    print('>>>>>>', sub_id)
    raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False,
                                      eog=['VEOG'])

    # create a dictionary of annotations
    desc = utils.change_annot_names(raw.annotations.description.copy(),
                                    enumerated=True)
    onsets = raw.annotations.onset.copy()
    annot_dict = {k: v for k, v in zip(desc, onsets)}

    # find the points where a switch between EC/EO happens
    switch_onsets, switch_pattern = utils.find_switch_onset_pattern(annot_dict)
    
    # segment raw data based on switch points
    raws = utils.segment_raw(raw, switch_onsets, switch_pattern, duration=60)
    
    # check if the segments are correct (only EC or EO in each segment)
    for s in ['EC', 'EO']:
        sanity_check[sub_id + '_' + s] = utils.check_segments(raws[s])
    
    raw_EO = raws['EO']
    raw_EO = raw_EO.pick(picks='eeg')
    
    raw_EC = raws['EC']
    raw_EC = raw_EC.pick(picks='eeg')
    del raws
    
    all_eo.append(raw_EO.resample(128).get_data())
    all_ec.append(raw_EC.resample(128).get_data())
    
# create a dataset
ds = xr.Dataset({'eye_open': (['subject', 'channel', 'timestep'], all_eo),
                 'eye_closed': (['subject', 'channel', 'timestep'], all_ec)},)

# add subject ids to the dataset and save
ds.coords['subject'] = subject_ids
ds.to_netcdf('eeg_eo_ec.nc5', engine='h5netcdf')
ds.close()

## Pre-processing

In [92]:
# Helper functions
# clamp
def clamp_eeg(data, axis=2, deviation=20):
    """Clamp EEG data proportional to the standard deviation of each channel."""
    # container for clamped data
    data_clamped = np.zeros_like(data)
    upper_dev = data.mean(axis=axis) + data.std(axis=axis) * deviation
    lower_dev = data.mean(axis=axis) - data.std(axis=axis) * deviation
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data_clamped[i, j, :] = np.clip(data[i, j, :], lower_dev[i, j], upper_dev[i, j])
    return data_clamped

def print_min_max(data, lower_dev, upper_dev, sub, channel):
    """Prints the min and max of a channel and checks if it would be clamped by the clamp_eeg function
    This is used to check if the clamp function is working as expected."""
    if data[sub, channel, :].min() < lower_dev[sub, channel] or data[sub, channel, :].max() > upper_dev[sub, channel]:
        print(f'Data: {sub}, {channel}: Min={data[sub, channel, :].min()}, Max={data[sub, channel, :].max()}',
              f'Bounderies: Lower={lower_dev[sub, channel]}, Higher={upper_dev[sub, channel]}')

### Baseline Correction

In [ ]:
# open dataset
da_name = 'eye_open'
ds = xr.open_dataset('eeg_eo_ec.nc5', engine='h5netcdf')
subject_ids = ds.subject.values
da = ds[da_name]
del ds

# convert to numpy array
data_eo = da.to_numpy()
del da

# baseline correction
data_eo = data_eo - data_eo[:, :, :int(128*0.5)].mean(axis=2, keepdims=True)

# save
data_eo = xr.DataArray(data_eo, dims=['subject', 'channel', 'timestep'])
data_eo.coords['subject'] = subject_ids
data_eo.to_netcdf('eeg_EO_BaseCorr.nc5', engine='h5netcdf')

### Normalization & Clamping

In [2]:
# open baseline corrected data
da = xr.open_dataarray('eeg_EO_BaseCorr.nc5', engine='h5netcdf')
da = da.sel(subject=da.subject.values[:5]).to_numpy()

# normalize
data = np.array(
          [RobustScaler().fit_transform(da[i, :, :]) for i in range(da.shape[0])]
        )
print(data.min(), data.max())

# clamp
data_clamped = clamp_eeg(data)

## Feature Extraction
### Connectivity

In [67]:
import simsimd

# compute cosine similarity
similarity_all = np.zeros((data_clamped.shape[0], 1830))  # 1830 is the lenght of the upper triangle of the similarity matrix
for sub in range(data_clamped.shape[0]):
    similarity = simsimd.cdist(data_clamped[sub, :, :], data_clamped[sub, :, :], metric='cosine')
    similarity = similarity[np.triu_indices(similarity.shape[0], k=1)]
    similarity_all[sub, :] = similarity